In [ ]:
import numpy as np
import pandas as pd
import sklearn as sk

from datetime import datetime, timedelta 

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as ex
import plotly.io as pio

import glob
import os

import seaborn
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

from joblib import dump, load
 
import tensorflow 
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, Input, GRU, LSTM
from keras.activations import relu, tanh, linear
from keras.layers import Dropout

from keras.callbacks import EarlyStopping
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import make_scorer

import json
#from interpolate import *

In [ ]:
from interpolate import interpolate
from plot import plot
from readIn import readIn
from plotHeatmap import plotHeatmap
from removeT_bett import removeT_bett
from trainDTR import trainDTR

In [ ]:
import scikeras
from scikeras.wrappers import KerasRegressor

In [ ]:
# Set parameters:
do_interpolation = False
scatter_mode = "lines"
plot_ipo = True

#Which models to use?
useDTR = False
use_sk_NN = False
use_keras_NN = False
use_keras_LSTM = True

# Nr of timesteps to consider for LSTM
timesteps = 10

# How to split the data into training, validation and test set?
split_evenly = True
split_by_coolant = False
split_by_profile = False

# Use pipeline?
usePipeline = True

# Plot?
plotNN = False
plotDTR = False

# Directories
#workdir = "G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\row_files\\combines\\smoothed\\With_cooling_feature_for_Model_training_5s\\validate_Data"
workdir = "X:\\KI Praktikum\\validate_Data"
#goaldir = "G:\\Innovations@HELLER\\DN\\KI\\Temperaturkompensation\\2021_Spindelwachstumskompensation_KI_HSU_SC63\\Messungen DC100 H5000 M57002\\row_files\\combines\\smoothed\\With_cooling_feature_for_Model_training_5s\\interpolated_Data_time"
goaldir = "X:\\KI Praktikum\\validate_Data_time_ipo"
dir_ipo = "X:\KI Praktikum\\validate_Data\ipo_time"

Read in several csv files and concatenate one large dataframe

In [ ]:
#df_merged = readIn(workdir, ipo=True, safeSingleCsv =False)
df_merged = readIn(dir_ipo, ipo=False, safeSingleCsv = False)
    

In [ ]:
df_merged = df_merged.drop(columns=["Unnamed: 0", "Unnamed: 0.2"]) #, "Unnamed: 0.3"
df_merged.to_csv("X:\\KI Praktikum\\validate_Data\\merged_df.csv")

Show correlations

In [ ]:
t_bett_abgezogen = True
z_interpoliert = True
show=False

plt = plotHeatmap(df_merged, show=show, t_bett_abgezogen=t_bett_abgezogen, z_interpoliert=z_interpoliert)
if plt != None:
    plt.show()

Show distribution of the data using a histogram

In [ ]:
hist = False
if hist:
    df_merged.hist(figsize=(15,20))

Split the entire dataframe into test, training and validation data

In [ ]:
filenames = df_merged["filename"].unique()
#print(filenames)

In [ ]:
if split_evenly:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch10_12_2022_trocken_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_082022_trocken_iso_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_11_2022_M8_iso_shifted_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_092022_trocken_prof3_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M121_iso__time_ipo.csv'
                        ]
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_trocken_iso_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M8_iso__time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof2_5s_time_ipo.csv',
                'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof2_5s_time_ipo.csv']

if split_by_coolant:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch11_12_2022_M8_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M127_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch12_11_2022_M8_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch13_01_2023_M127_iso_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch15_11_2022_M7_iso_shifted_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M121_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M121_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_082022_M8_iso_5s_time_ipo.csv']
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_11_2022_M8_iso_shifted_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_01_2023_M8_M121_isoteil_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof4_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_12_14_10_M8_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_13_09_30_M7_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_17_17_04_M127_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch2023_01_18_06_00_M121_iso__time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_11_2022_M8_prof3_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_M8_iso_time_ipo.csv']
if split_by_profile:
    validation_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof2_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch19_11_2022_M8_prof4_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch20_11_2022_M8_prof3_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof1_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch21_092022_trocken_prof2_5s_time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch18_01_2023_M8_M127_iso__time_ipo.csv',
                        'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_11_2022_trocken_iso_time_ipo.csv']
    test_files = ['X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch22_092022_trocken_prof3_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof1_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch24_092022_trocken_prof2_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch25_092022_trocken_prof3_5s_time_ipo.csv',
                    'X:\\KI Praktikum\\validate_Data\\ipo_time\\filtered_interpolated_Versuch17_01_2023_M127_iso__time_ipo.csv']
    
df_train = df_merged[df_merged["filename"].isin(validation_files) == False]
#print("w/o validation " + str(df_train.info()))
df_train = df_train[df_train["filename"].isin(test_files) == False]
df_validation = df_merged[df_merged["filename"].isin(validation_files) == True]
df_test = df_merged[df_merged["filename"].isin(test_files) == True]
#print("w/o test " + str(df_train.info()))

df_train["welle_z_ipo_time"] = 1000*df_train["welle_z_ipo_time"]
df_validation["welle_z_ipo_time"] = 1000*df_validation["welle_z_ipo_time"]
df_test["welle_z_ipo_time"] = 1000*df_test['welle_z_ipo_time']

Set up a preprocessing pipeline and preprocess data

In [ ]:
y_train = df_train["welle_z_ipo_time"]
parameters = ["t_bett", "t_motor", "t_spindle", "M8", "M121", "M127", "M7"]
x_train = df_train[parameters].to_numpy()

y_validation = df_validation["welle_z_ipo_time"]
x_validation = df_validation[parameters].to_numpy()

y_test = df_test["welle_z_ipo_time"]
x_test = df_test[parameters].to_numpy()


In [ ]:
seq_steps = [("remove_t_bett", FunctionTransformer(removeT_bett)), ("standardize_values", StandardScaler()), ("pca", PCA(6))]
pipeline = Pipeline(steps=seq_steps)

#print(np.argwhere(np.isnan(x_train)))
if usePipeline:
    x_transform = pipeline.fit_transform(x_train)
    x_validation = pipeline.transform(x_validation)
    x_test = pipeline.transform(x_test)
else:
    x_transform = x_train

In [ ]:
pathname = "X:\\KI Praktikum\\pipeline.p"
dump(pipeline, pathname)

In [ ]:
if usePipeline:
    newpipeline = load(pathname)
    newpipeline.steps[2][1].explained_variance_ratio_

Train a Decision Tree Regressor

In [ ]:
if useDTR:
    score_dtr, figTrain, figValidation = trainDTR(x_transform, y_train, x_validation, y_validation, scatter_mode)
    if plotDTR:
        figTrain.show()
        figValidation.show()

Train a neural network using cross-validation

In [ ]:
if use_sk_NN:
    use_train_and_validation=True

    if use_train_and_validation:
        x_cv = np.concatenate((x_transform, x_validation))
        y_cv = np.concatenate((y_train, y_validation))

    else:
        x_cv = x_transform
        y_cv = y_train
    nn = MLPRegressor(solver= 'adam', n_iter_no_change = 15, random_state=0, verbose=True)

    parameters = {'hidden_layer_sizes': [(5, 5, 5, 5, 5, 5), (10, 10), (3, 3, 3, 3), (15, 15)],
                'activation': ['tanh', 'relu'],
                'max_iter': [500, 1000, 5000, 6000, 7000]}

    randCV = RandomizedSearchCV(estimator=nn, param_distributions=parameters, cv =5, n_iter = 4, n_jobs=-1)
    randCV.fit(x_cv, y_cv)

    # print(randCV.best_estimator_)
    # print('score: ', randCV.score)
    # print('best parameters: ', randCV.best_params_)

In [ ]:
if use_sk_NN:
    randCV.best_params_

In [ ]:
if use_sk_NN:
    nn = randCV.best_estimator_
    nn.fit(x_transform, y_train)
    score_nn = nn.score(x_validation, y_validation)

In [ ]:
if use_sk_NN and useDTR:
      print("Score Decision Tree Regressor: ", score_dtr, "\n",
      "Score Neural Network: ",  score_nn)

In [ ]:
if plotNN:
    nn_pred_train = nn.predict(x_transform)

    fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter( y= y_train[100000:120000], name= 'y-values', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter( y= nn_pred_train[100000:120000], name= 'y-predicted', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'y-train', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Predictions for training data neural network")
    #fig.write_image("X:\\KI Praktikum\\validate_Data\\Bilder\\Ungleiche_Datenverteilung\\NN_training_coolant.jpeg")
    fig.show()

In [ ]:
if plotNN:
    nn_pred_validation = nn.predict(x_validation)

    fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter( y= y_validation, name= 'y-values', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter( y= nn_pred_validation, name= 'y-predicted', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'y-train', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Predictions for validation data neural network")
    fig.show()

In [ ]:
if use_keras_NN:
    earlyStop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto') 

    model = Sequential()
    model.add(Dense(units=3, activation = relu))
    model.add(Dense(units=2, activation = relu))
    model.add(Dense(units=1, activation = linear))

    model.compile(optimizer = "Nadam", loss = ks.losses.mean_absolute_error)

    summary = model.fit(x_transform, y_train, verbose=1, batch_size = 200, epochs=100, validation_data = (x_validation, y_validation), callbacks = [earlyStop])


In [ ]:
if use_keras_NN:
    train_loss = summary.history["loss"]
    validation_loss = summary.history["val_loss"]

    model_pred_validation = model.predict(x_validation)

    fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

    fig.add_trace(go.Scatter( y= train_loss, name= 'training loss', mode= scatter_mode), row= 1, col= 1)
    fig.add_trace(go.Scatter( y= validation_loss, name= 'validation loss', mode= scatter_mode), row= 1, col= 1)
    fig.update_yaxes(title_text= 'loss', row= 1, col= 1)

    fig.update_layout(height=600, width=1200, title_text="Training loss curve")
    fig.show()

In [ ]:
#model.save("X:\\KI Praktikum\\validate_Data\\models\\keras_NN_321_Nadam.h5")

In [ ]:
use_train_and_validation=True

if use_train_and_validation:
    x_cv = np.concatenate((x_transform, x_validation))
    y_cv = np.concatenate((y_train, y_validation))

else:
    x_cv = x_transform
    y_cv = y_train

In [ ]:
def get_mlp_model(layers=[5, 5, 5], dropout=0, learnRate=0.001, activation = "relu", optimizer="Nadam", first_activ = "relu"):
    model = Sequential()
    first = True
    for layer in layers:
        if first:
            model.add(Dense(layer, activation=first_activ, input_dim=6))
            first = False
        else:
            model.add(Dense(layer, activation=activation))
        model.add(Dropout(dropout))


    model.add(Dense(1, activation="linear"))
    # compile the model
    model.compile(
        optimizer=optimizer,
        loss=ks.losses.mean_absolute_error)
    # return compiled model
    return model

In [ ]:
if use_keras_NN:
    model = KerasRegressor(build_fn=get_mlp_model, verbose=1, layers=[5, 5, 5], dropout=0, activation = "relu", optimizer="Nadam", first_activ = "relu")     
                    #, first_rec_activ="sigmoid", rec_activ="sigmoid"
    model.fit(x_transform, y_train)

In [ ]:
if use_keras_NN:
    grid = dict( 
        layers = [ [500, 500, 300], [100, 150, 50], 
                [700, 500, 300], [700, 700, 500]],    #[500, 400, 200], [700, 700, 500, 300], [700, 700, 500, 100],[25, 25, 25, 20, 20, 20, 10, 10],  [75, 75, 50, 50, 50, 40, 40, 20]
        dropout = [0, 0.1, 0.3, 0.2],
        first_activ = ["relu", "tanh"],
        activation = ["relu", "tanh"],
        #first_rec_activ = ["sigmoid", "tanh"],
        #rec_activ = ["sigmoid", "tanh"],
        optimizer = ["Nadam", "Adam"]
    )

    searcher = RandomizedSearchCV(estimator=model, n_jobs=1, cv=3,
        param_distributions=grid, scoring='neg_mean_absolute_error', n_iter = 5)
    searchResults = searcher.fit(x_cv, y_cv)

In [ ]:
if use_keras_NN:	
	bestScore = searchResults.best_score_
	bestParams = searchResults.best_params_
	print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

	bestModel = searchResults.best_estimator_
	bestModel.fit(x_transform, y_train)
	score = bestModel.score(x_validation, y_validation)
	print("score: {:.2f}%".format(score * 100))

In [ ]:
if use_keras_NN:
    best_pred = bestModel.predict(x_validation)
    error_avg = np.mean(abs(y_validation - best_pred))
    error_max = max(abs(y_validation - best_pred))
    print("max: ", error_max)
    print("avg: ", error_avg)

In [ ]:
#sess = tensorflow.compat.v1.Session(config=tensorflow.compat.v1.ConfigProto(log_device_placement=True))

Reshape data for the training of a LSTM

In [ ]:
print(x_transform.shape)
print(df_train.shape)
print(len(df_train["filename"].unique()))
print("longest sample: ", max(df_train["filename"].value_counts()))
print("shortest sample: ", min(df_train["filename"].value_counts()))

In [ ]:
def arr2d_to_arr3d(x_2d, y, window, stride):
    
    x_2d = x_2d[:-1, :]
    y = np.roll(y, -1, axis=0)
    y = y[window-1:-1]

    isample = window-1
    x_3d = np.zeros((1, window, x_2d.shape[1]))
    # for each sample in the data frame create the corresponding batch consisting of the earlier samples
    # loop for each sample
    while isample < x_2d.shape[0]:
        i = 0
        arr_sample =np.zeros((1, window, x_2d.shape[1]))
        # loop to create frame of previous and current samples
        while i > -window:
            singleSample = x_2d[isample+i, :] 
            arr_sample[0, window-1+i, :] = singleSample
            # print("arr_sample: ", arr_sample)
            i= i-1
        
        if isample == window-1:
            x_3d = arr_sample
        else:
            x_3d = np.concatenate((x_3d, arr_sample), axis = 0)

        isample += stride
    return x_3d, y


In [ ]:
# generateDataSource() -> 3D array von den Daten
### window  represents time period by each entry in the buffer
### Shift represents the jump from value to next one in the buffer
### sampling rate

def generateDataSource(signal_input=None, input_columns: list = [], output_length: int = 1, signal_output=None, window=1, shift=1, sample_rate=1):
    #subsequence_len= (window -1) *shift + 1
    subsequence_len= (window) *shift
    Signal_Length = signal_input.shape[0]
    num_samples = 1 + int((Signal_Length - subsequence_len) / sample_rate)
    x = np.zeros(shape=(num_samples, window, signal_input.shape[1]))
    y = np.zeros(shape=(num_samples, output_length, 1))
    for i in range(num_samples):
        x[i] = np.asarray([signal_input[i*sample_rate + j * shift] for j in range(0,window)])
        y[i] = signal_output[i*sample_rate + (window-1) * shift :i*sample_rate+ (window-1) * shift + output_length]
    return x, y

In [ ]:
# window = 3
# stride = 1
# x = np.array([[0, 1, 2, 3, 4],
#      [5, 6, 7, 8, 9],
#      [10, 11, 12, 13, 14],
#      [15, 16, 17, 18, 19],
#      [20, 21, 22, 23, 24],
#      [25, 26, 27, 28, 29],
#      [30, 31, 32, 33, 34],
#      [35, 36, 37, 38, 39]])
# y = np.array([1, 2, 3, 4, 5, 6, 7, 8])

# x_3d, y = arr2d_to_arr3d(x, y, window, stride)
# print(x_3d, y)

In [ ]:
#x_3d.tolist()

In [ ]:
# # Using my code
# window = 10
# stride = 1

# if use_keras_LSTM:
#     x_cv = np.concatenate((x_transform, x_validation))
#     y_cv = np.concatenate((y_train, y_validation))
    
#     x_cv, y_cv = arr2d_to_arr3d(x_cv, y_cv, 10, 1)


3D arrays mit verschiedenen Window-sizes als json abspeichern

In [ ]:
# Using Mustafas function
def SaveJsonWindows(filenames, window):
    i = 0
    for w in windows:
        x_cv, y_cv = generateDataSource(signal_input = np.concatenate((x_transform, x_validation)),
                                        signal_output=np.concatenate((y_train, y_validation)), 
                                        window = w, shift=1, sample_rate=1)
        out_x = open(filenames[i][0], mode="x")
        json.dump(x_cv.tolist(), out_x)
        out_x.close()
        out_y = open(filenames[i][1], mode="x")
        json.dump(y_cv.tolist(), out_y)
        out_y.close()
        i += 1
    

Json File auslesen und als 3D Array verwenden

In [ ]:
def OpenJsontoArr(path):
    file = open(path)           #"X:\\KI Praktikum\\validate_Data\\3darray_x_cv.json"
    x_3d = json.load(file)
    file.close()
    x_3d = np.asarray(x_3d)
    return x_3d

In [ ]:
out_x = open("X:\\KI Praktikum\\validate_Data\\3darray_x_cv.json", mode="x")
json.dump(x_cv.tolist(), out_x)
out_x.close()
out_y = open("X:\\KI Praktikum\\validate_Data\\3darray_y_cv.json", mode="x")
json.dump(y_cv.tolist(), out_y)
out_y.close()

LSTM Modell definieren

In [ ]:
from keras.layers import Input

In [ ]:
def get_lstm_model(layers=[5, 5, 5], dropout=0, activation = "relu", input_shape = (1, 1)):
    
    model = Sequential()
    model.add(Input(input_shape))
    
    for i in range(len(layers)):
        if i == len(layers)-1:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=False))
        else:
            model.add(LSTM(layers[i], stateful = False, dropout=dropout, activation = activation, return_sequences=True))
        #model.add(Dropout(dropout))

    model.add(Dense(1, activation="linear"))
    # compile the model
    model.compile(
        loss=ks.losses.mean_absolute_error,
        optimizer = "Nadam")
    # return compiled model
    return model

In [ ]:
# # Example Code Mustafa
# def build_Model_LSTM(input_shape, activation_Function = selu,
#     dropout = 0.0,
#     l1_v = 0.00,
#     l2_v = 0.00,
#     structure= [],
#     optimizer= 'Nadam'):
    
#     #structure=  [n_units for i in range(0,n_hidden_layers)]#[50,50,40,40,30,30,20]##  
#     unroll = False
#     kernal_init = RandomNormalV2()#'he_normal'#RandomUniform()  #
#     model = Sequential()
#     model.add(Input(shape=input_shape))
#     # model.add(Dense(50, activation=activation_Function))
#     model.add(Dense(20, activation=activation_Function))
#     model.add(Dense(10, activation=activation_Function))
#     for i in range(1,len(structure)+1):
#         layer_size= structure[i-1]
#         if i == len(structure):
#             model.add(LSTM(layer_size,stateful= False,return_sequences=False,unroll=unroll,kernel_initializer= kernal_init, dropout=dropout, kernel_regularizer=L1L2(l1=l1_v, l2=l2_v)))
#         else:
#             model.add(LSTM(layer_size,stateful= False, return_sequences=True,unroll=unroll, kernel_initializer= kernal_init,dropout=dropout,  kernel_regularizer=L1L2(l1=l1_v, l2=l2_v)))
#         #model.add(BatchNormalization())
#         model.add(Activation(activation=activation_Function))
#         #model.add(BatchNormalization())
#         #model.add(PReLU())
#     model.add(Dense(10, activation=activation_Function))
#     model.add(Dense(5, activation=activation_Function))
#     model.add(Dense(1, activation='relu'))
#     model.compile(optimizer=optimizer, loss= mean_squared_error, metrics=[ max_loss])#'Adagrad'
#     return model

In [ ]:
    # x_cv = np.concatenate((x_train_3d, x_validation_3d), 0)
    # y_cv = np.concatenate((y_train, y_validation), 0)

In [ ]:
lstmmodel = get_lstm_model(layers=[5, 5, 5], dropout=0, activation = "relu",      
                    input_shape = (10,6))
x_lstm_train, x_lstm_test, y_lstm_train, y_lstm_test = train_test_split(x_cv, y_cv, shuffle = False, test_size=0.1)


In [ ]:
summary = lstmmodel.fit(x_lstm_train, y_lstm_train, batch_size=500, workers = 10, use_multiprocessing=True, validation_data=(x_lstm_test, y_lstm_test), epochs = 500, stop)

Training loss curve

In [ ]:
# # Plot loss curve
train_loss = summary.history["loss"]
validation_loss = summary.history["val_loss"]

fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

fig.add_trace(go.Scatter( y= train_loss, name= 'training loss', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter( y= validation_loss, name= 'validation loss', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'loss', row= 1, col= 1)

fig.update_layout(height=600, width=1200, title_text="Training loss curve")
fig.show()

Difference between predicted and meassured y - Test

In [ ]:
# #y_train_pred = lstmmodel.predict(x_lstm_train)
y_test_pred = lstmmodel.predict(x_lstm_test)

fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

fig.add_trace(go.Scatter(y= y_test_pred.flatten(), name= 'y-predicted', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter(y= y_lstm_test, name= 'y-test', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'welle_z', row= 1, col= 1)

fig.update_layout(height=600, width=1200, title_text="Difference between predicted and meassured y - Test")
fig.show()

Difference between predicted and meassured y - Training

In [ ]:
# y_train_pred = lstmmodel.predict(x_lstm_train)
#y_test_pred = lstmmodel.predict(x_lstm_test)

fig= make_subplots(rows=1, cols=1,shared_xaxes= True, print_grid= True)

fig.add_trace(go.Scatter( y= y_train_pred.flatten(), name= 'y-predicted', mode= scatter_mode), row= 1, col= 1)
fig.add_trace(go.Scatter( y= y_lstm_train, name= 'y-train', mode= scatter_mode), row= 1, col= 1)
fig.update_yaxes(title_text= 'welle_z', row= 1, col= 1)

fig.update_layout(height=600, width=1200, title_text="Difference between predicted and meassured y - Training")
fig.show()

In [ ]:
error_avg = np.mean(abs(y_lstm_test - y_test_pred.flatten()))
error_max = max(abs(y_lstm_test - y_test_pred.flatten()))
print("max: ", error_max)
print("avg: ", error_avg)

Hyperparameter suchen für Daten mit verschiedenen Batchsizes

In [ ]:
windows = [5, 10, 15, 20]
filenames = [["X:\\KI Praktikum\\validate_Data\\3darray_window5_x.json", "X:\\KI Praktikum\\validate_Data\\3darray_window5_y.json"], 
            ["X:\\KI Praktikum\\validate_Data\\3darray_window10_x.json", "X:\\KI Praktikum\\validate_Data\\3darray_window10_y.json"],
            ["X:\\KI Praktikum\\validate_Data\\3darray_window15_x.json", "X:\\KI Praktikum\\validate_Data\\3darray_window15_y.json"],
            ["X:\\KI Praktikum\\validate_Data\\3darray_window20_x.json", "X:\\KI Praktikum\\validate_Data\\3darray_window20_y.json"]]

modelpaths = ["X:\\KI Praktikum\\validate_Data\\modelle2023-08-25\\window5.h5",
                "X:\\KI Praktikum\\validate_Data\\modelle2023-08-25\\window10.h5",
                "X:\\KI Praktikum\\validate_Data\\modelle2023-08-25\\window15.h5",
                "X:\\KI Praktikum\\validate_Data\\modelle2023-08-25\\window20.h5"]

SaveJsonWindows(filenames=filenames, window=windows)
i = 0

for file in filenames:
    print(file[0])
    x_cv = OpenJsontoArr(file[0])
    y_cv = OpenJsontoArr(file[1])
    y_cv = y_cv.flatten()
    
    earlyStop = EarlyStopping(monitor='loss', patience=10, verbose =1, mode = "auto")
    model = KerasRegressor(build_fn=get_lstm_model, verbose=1, layers=[5, 5, 5], dropout=0, activation = "relu", 
                            input_shape = (windows[i], 6))
    model.fit(x_cv, y_cv)
    
    grid = dict(layers = [[2], [5], [2, 2], [2, 5], [5, 2], [5, 5], [10, 5], [10, 10], [2, 2, 2]],
                dropout = [0, 0.1, 0.3, 0.2],
                activation = ["relu", "tanh", "selu", "elu"])

    searcher = RandomizedSearchCV(estimator=model, n_jobs=1, cv=3, param_distributions=grid, scoring='neg_mean_absolute_error', n_iter = 5)
    searchResults = searcher.fit(x_cv, y_cv, batch_size = 500, epochs=500, callbacks = [earlyStop], use_multiprocessing = True)

    bestScore = searchResults.best_score_
    bestParams = searchResults.best_params_
    bestModel = searchResults.best_estimator_
    print("[INFO] best score is {:.2f} using {}".format(bestScore, bestParams))
    bestModel.save(modelpaths[i])

    best_pred = bestModel.predict(x_cv)
    difference = y_cv - best_pred
    error_avg = np.mean(abs(difference))
    error_max = max(abs(difference))
    print("max: ", error_max)
    print("avg: ", error_avg)
    
    i += 1

In [ ]:
if use_keras_LSTM:
    model = KerasRegressor(build_fn=get_lstm_model, verbose=1, layers=[5, 5, 5], dropout=0, activation = "relu",      
                    input_shape = (10,6))
    model.fit(x_cv, y_cv)

In [ ]:
if use_keras_LSTM:
    grid = dict( 
        layers = [[500, 500, 300], [100, 150, 50], 
                [700, 500, 300], [700, 700, 500], [500, 400, 200], 
                [700, 700, 500, 300], [700, 700, 500, 100],
                [25, 25, 25, 20, 20, 20, 10, 10],  [75, 75, 50, 50, 50, 40, 40, 20]],
        dropout = [0, 0.1, 0.3, 0.2],
        activation = ["relu", "tanh", "selu", "elu"],
    )


    searcher = RandomizedSearchCV(estimator=model, n_jobs=1, cv=3,
        param_distributions=grid, scoring='neg_mean_absolute_error', n_iter = 5)
    searchResults = searcher.fit(x_cv, y_cv)

In [ ]:
if use_keras_LSTM:	
	bestScore = searchResults.best_score_
	bestParams = searchResults.best_params_
	print("[INFO] best score is {:.2f} using {}".format(bestScore,
	bestParams))

	bestModel = searchResults.best_estimator_
	bestModel.fit(x_3d, y_train)
	score = bestModel.score(x_validation, y_validation)
	print("score: {:.2f}%".format(score * 100))

In [ ]:
if use_keras_LSTM:
    best_pred = bestModel.predict(x_validation)
    error_avg = np.mean(abs(y_validation - best_pred))
    error_max = max(abs(y_validation - best_pred))
    print("max: ", error_max)
    print("avg: ", error_avg)